In [1]:
import psycopg2
import pandas as pd

# Prepare the Database

Run `docker-compose up` to get Postgres up and running.

In [2]:
credentials = "dbname='astronomy' user='user' host='localhost' password='pass'"
conn = psycopg2.connect(credentials)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY,
  t_eff INTEGER,
  radius FLOAT
);


COPY Star (kepler_id, t_eff, radius) FROM '/data/data1/stars.csv' CSV;
""")
conn.commit()

# Task 1
As a warm-up, add the following stars to the existing Star table:

| kepler_id | t_eff | radius |
|---|---|---|
| 7115384 | 3789 | 27.384 |
| 8106973 | 5810 | 0.811 |
| 9391817 | 6200 | 0.958 |

In [3]:
pd.read_sql("""
INSERT INTO Star (kepler_id, t_eff, radius) VALUES
  (7115384, 3789, 27.384),
  (8106973, 5810, 0.811),
  (9391817, 6200, 0.958);

SELECT * FROM Star;
""", conn)

,kepler_id,t_eff,radius
0,10341777,6302,0.815
1,11296798,6335,3.523
2,3836450,5160,0.784
3,4483235,8782,1.965
4,6590362,5926,0.887
5,7031208,6319,1.032
6,7115384,3789,27.384
7,8106973,5810,0.811
8,9391817,6200,0.958
